# Classification

In [2]:
import tensorflow as tf
import numpy as np
import helper

## On Data

These data are the results of a chemical analysis of wines grown in the same region in Italy
but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines. 

I think that the initial data set had around 30 variables, but for some reason I only have the 13 dimensional version. I had a list of what the 30 or so variables were, but a.) I lost it, and b.), I would not know which 13 variables are included in the set. 

The attributes are (dontated by Riccardo Leardi, riclea '@' anchem.unige.it ) 

1. Alcohol 
2. Malic acid 
3. Ash 
4. Alcalinity of ash 
5. Magnesium 
6. Total phenols 
7. Flavanoids 
8. Nonflavanoid phenols 
9. Proanthocyanins 
10. Color intensity 
11. Hue 
12. OD280/OD315 of diluted wines 
13. Proline 

In [3]:
data_x, data_y = helper.read_wine_data()

In [5]:
print(data_x.shape)
print(data_y.shape)

(178, 13)
(178, 3)


In [6]:
data_x[0]

array([ 0.787     ,  0.186     ,  0.45500001,  0.278     ,  0.28299999,
        0.57599998,  0.41999999,  0.245     ,  0.495     ,  0.292     ,
        0.45500001,  0.85000002,  0.54000002], dtype=float32)

In [7]:
data_y[0]

array([1, 0, 0])

In [18]:
train_data_x, train_data_y = data_x[:150], data_y[:150]
test_data_x, test_data_y = data_x[150:], data_y[150:]

We need a classifier, that maps the feature vector to a class of wine (1, 2 or 3)

## GRAPH

In [8]:
num_features = data_x.shape[-1]
num_classes = data_y.shape[-1]

- placeholders

In [9]:
tf.reset_default_graph()
x_ = tf.placeholder(shape=[num_features], dtype=tf.float32, name='x')
y_ = tf.placeholder(shape=[num_classes], dtype=tf.float32, name='y')

* parameters/variables

In [10]:
W = tf.get_variable(dtype=tf.float32, shape=[num_features, num_classes], name='W',
                   initializer=tf.contrib.layers.xavier_initializer())
b = tf.get_variable(dtype=tf.float32, shape=[num_classes], name='b',
                    initializer=tf.constant_initializer(0.))

- operations

In [16]:
logits = tf.matmul(tf.reshape(x_, [1, num_features]), W) + b
probs = tf.nn.softmax(logits)
prediction = tf.argmax(probs, axis=1)
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_)

- optimization

In [17]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(loss)

## Training

In [21]:
epochs = 1000

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(epochs):
    avg_loss = 0.
    for batch_x, batch_y in zip(train_data_x, train_data_y):
        _, loss_value = sess.run([train_op, loss], feed_dict = {
            x_ : batch_x,
            y_ : batch_y
        })
        avg_loss += loss_value
    if (i and i%100 == 0) or i == epochs-1:
        print('loss : ', avg_loss/len(train_data_x))

loss :  [ 0.03437493]
loss :  [ 0.02117923]
loss :  [ 0.01562519]
loss :  [ 0.01247798]
loss :  [ 0.01042869]
loss :  [ 0.00897978]
loss :  [ 0.00789723]
loss :  [ 0.00705567]
loss :  [ 0.00638163]
loss :  [ 0.0058339]


## Wine Tasting Time!

In [25]:
j = np.random.randint(0, len(test_data_x)-1)
test_xj, test_yj = test_data_x[j], test_data_y[j]
predicted_value, confidence = sess.run([prediction, probs], feed_dict= {
    x_ : test_xj
})
print('Real Wine Class :', np.argmax(test_yj))
print('Predicted Wine Class : ', predicted_value[0])
print('\t with confidence of {}%'.format(confidence[0][predicted_value]*100))

Real Wine Class : 1
Predicted Wine Class :  1
	 with confidence of [ 99.99989319]%
